# Traveling Salesman Problem

<a target="_blank" href="https://colab.research.google.com/github/arthurrichards77/smply/blob/master/travelling_salesman.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This example shows how an optimization can be used to solve a Traveling Salesman Problem over a roadmap.

First we'll need some libraries including the [PuLP linear programming library](https://coin-or.github.io/pulp/).

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
%pip install pulp
from pulp import *

## Problem definition

Start by choosing 20 random locations.

In [ ]:
n=20
point_options = [[x,y] for x in range(10) for y in range(10)]
visit_points = np.array(random.sample(point_options,n))

plt.plot(visit_points[:,0], visit_points[:,1],'g.')
for pp in range(n):
    plt.annotate(str(pp),(visit_points[pp,0], visit_points[pp,1]))
plt.show()

Now find the distances between them.  For simplicyt, I'll use straight line between all involved.  You could have `inf` values in here as well though - for example, the adjacency graph from any of the roadmap methods could be used.

In [ ]:
from math import sqrt
distance = np.zeros((20,20))
for ii in range(20):
  for jj in range(ii,20):
    distance[ii,jj] = sqrt(sum((visit_points[ii,:]-visit_points[jj,:])**2))
    distance[jj,ii]=distance[ii,jj]
distance

## The TSP bit

Now we've built the roadmap, instead of running an A-to-B search like Dijkstra's on it, let's find the shortest path that visits all points.

We solve this using integer linear optimization, using a library called PuLP to model and solve the problem.  First, get a list of all the possible moves or "links" and create a new optimization problem with a binary decision variable for each, $X(i,j) = \{0,1\} \ \forall i \ne j$.  We will intepret $X(i,j)=1$ to mean that the shortest path goes from $i$ to $j$.

In [ ]:
n = len(visit_points)
links = [(i,j) for i in range(n) for j in range(n) if j!=i]

prob = LpProblem('tsp',LpMinimize)
x = LpVariable.dicts("x",links,0,1,LpInteger)

Define the objective to be the length of the path, which is the sum of all the decision variables weighted by the length of each link, $\sum_{i,j \ i \ne j} D(i,j)X(i,j)$. 

In [ ]:
prob.setObjective(sum([distance[i,j]*x[i,j] for (i,j) in links]))


The main constraints are that the path must depart from every node exactly once $\sum_{j \in \mathcal{N}(i)} X(i,j) = 1 \ \forall i$ and also arrive at every node exactly once $\sum_{i  \in \mathcal{N}(j)} X(i,j)=1 \ \forall j$ where $\mathcal{N}(n)$ denotes the nodes connected to node $n$, in our case every other node.

In [ ]:
for i in range(n):
    prob += (sum(x[ic,j] for (ic,j) in links if ic==i)==1)
for j in range(n):
    prob += (sum(x[i,jc] for (i,jc) in links if jc==j)==1)

In [ ]:
prob.solve()
print("Status", LpStatus[prob.status], "with cost", prob.objective.value())
[(i,j) for (i,j) in links if x[i,j].value()==1]

In [ ]:
for (ii,jj) in links:
    if x[ii,jj].value()==1.:
        plt.plot([visit_points[ii,0],visit_points[jj,0]],[visit_points[ii,1],visit_points[jj,1]],'m-')
plt.plot(visit_points[:,0], visit_points[:,1],'g.')
for pp in range(n):
    plt.annotate(str(pp),(visit_points[pp,0], visit_points[pp,1]))
plt.show()

Well that's a bit rubbish.  Yes, every node has one arrival and one departure, but apparently not all by the same salesman.

One last issue: we can satisfy the constraints so far with two (or more) disconnected circuits or _subtours_, provided they cover all the nodes between them.  This isn't what we're looking for.  There are various approaches to solving _the subtour problem_ but here we'll use extra variables $V(i)$ to extract the ordering of the visits, with a constraint ensuring $V(j) = V(i)+1$ if $X(i,j)=1$ for every $j>0$.  A subtour not including node $0$ would contradict this constraint as the ordering constraints would be circular: $3>2>1>3$.

In [ ]:
v = LpVariable.dicts("v",range(n),0,n)
for (i,j) in links:
    if j!=0:
        prob += (v[j]>=v[i]+1-n*(1-x[i,j])) 

We can take a look at the problem and while it is pretty ugly, look closely and you can identify the elements of the model.

In [ ]:
prob

Now we simply as PuLP to solve it and look at the result.

In [ ]:
prob.solve()
print("Status", LpStatus[prob.status], "with cost", prob.objective.value())
[(i,j) for (i,j) in links if x[i,j].value()==1]

Plot the results and we should have an efficient tour round all the points.

In [ ]:
for (ii,jj) in links:
    if x[ii,jj].value()==1.:
        plt.plot([visit_points[ii,0],visit_points[jj,0]],[visit_points[ii,1],visit_points[jj,1]],'m-')
plt.plot(visit_points[:,0], visit_points[:,1],'g.')
for pp in range(n):
    plt.annotate(str(pp),(visit_points[pp,0], visit_points[pp,1]))
plt.show()